In [4]:
import pandas as pd
import numpy as np

stores = pd.read_csv("datos/stores.csv")
providers = pd.read_csv("datos/providers1.csv")
visits = pd.read_csv("datos/visits_1.csv")
answer = pd.read_csv("datos/answer.csv")
types = pd.read_csv("datos/types.csv")
# Renombrar columnas con sufijo por tabla
stores = stores.add_suffix("_store")
providers = providers.add_suffix("_provider")
visits = visits.add_suffix("_visit")
answer = answer.add_suffix("_answer")
type = types.add_suffix("_type")
# Unir tablas
df_filtrado = visits.merge(
    answer, left_on="id_visit", right_on="visit_id_answer", how="left"
)
df_filtrado = df_filtrado.merge(
    stores, left_on="store_id_visit", right_on="id_store", how="left"
)
df_filtrado = df_filtrado.merge(
    providers, left_on="user_id_visit", right_on="id_user_provider", how="left"
)
df_filtrado = df_filtrado.merge(
    type, left_on="type_id_visit", right_on="id_type", how="left"
)
df_filtrado.head()


,created_at_visit,Cruce_visit,DiaMes_visit,ended_at_visit,fecha_visit,Fecha_Menos6h_visit,FechaDia_visit,FechaHora_Menos6h_visit,id_visit,Incidencia válida_visit,...,store_region_store,code_provider,email_provider,id_proveedor_provider,name_provider,status_provider,full_name_user_provider,id_user_provider,id_type,name_type
0,2025-07-21 21:25:09,48,04 sep,2025-09-04 14:11:30,2025-09-04 00:00:00,2025-09-04 08:11:30,2025-09-04 00:00:00,2025-09-04 08:11:30,48,True,...,Reyes,CM,vaimi2599@gmail.com,3,CORPORATIVO MDA,Active,CORPORATIVO MDA,7,1,Lona
1,2025-07-21 21:25:09,48,04 sep,2025-09-04 14:11:30,2025-09-04 00:00:00,2025-09-04 08:11:30,2025-09-04 00:00:00,2025-09-04 08:11:30,48,True,...,Reyes,CM,vaimi2599@gmail.com,3,CORPORATIVO MDA,Active,CORPORATIVO MDA,7,1,Lona
2,2025-07-21 21:25:09,48,04 sep,2025-09-04 14:11:30,2025-09-04 00:00:00,2025-09-04 08:11:30,2025-09-04 00:00:00,2025-09-04 08:11:30,48,True,...,Reyes,CM,vaimi2599@gmail.com,3,CORPORATIVO MDA,Active,CORPORATIVO MDA,7,1,Lona
3,2025-07-21 21:25:09,48,04 sep,2025-09-04 14:11:30,2025-09-04 00:00:00,2025-09-04 08:11:30,2025-09-04 00:00:00,2025-09-04 08:11:30,48,True,...,Reyes,CM,vaimi2599@gmail.com,3,CORPORATIVO MDA,Active,CORPORATIVO MDA,7,1,Lona
4,2025-07-21 21:25:09,48,04 sep,2025-09-04 14:11:30,2025-09-04 00:00:00,2025-09-04 08:11:30,2025-09-04 00:00:00,2025-09-04 08:11:30,48,True,...,Reyes,CM,vaimi2599@gmail.com,3,CORPORATIVO MDA,Active,CORPORATIVO MDA,7,1,Lona


In [5]:
#Eliminar duplicados de id_vist y traer la info

tabla = df_filtrado.drop_duplicates(subset=["id_visit"])[[
        "id_visit",
        "store_sap_store",
        "store_name_store",
        "store_zone_store",
        "store_region_store",
        "name_type",
        "FechaHora_Menos6h_visit",
        "status_visit",
        "name_provider"

]]
tabla.head()


,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider
0,48,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,CORPORATIVO MDA
5,55,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,CORPORATIVO MDA
10,2555,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA
12,2556,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA
17,2560,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA


In [6]:
tabla

,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider
0,48,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,CORPORATIVO MDA
5,55,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,CORPORATIVO MDA
10,2555,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA
12,2556,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA
17,2560,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA
...,...,...,...,...,...,...,...,...,...
6987,10636,101002602,MISC ROSITA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 10:30:06,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA
6991,10637,120290750,MISC TU Y YO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-29 10:04:14,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA
6996,10638,120301264,TIENDA BEATRIZ,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-31 15:03:23,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA
7000,10639,120355365,TIENDA NANCY,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-29 19:36:44,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA


In [7]:
# Calculo tiene incidencia
incidencias = df_filtrado[df_filtrado["sectionid_answer"].isin([2,5])] #Se utiliza isin para filtrar los valores de la encuesta que tengan ese valor
incidencias = incidencias.groupby("id_visit").size().reset_index(name="count_incidencias") #cuenta la cantidad de incidencias que hay en cada visita y lo vuelve dataframe
incidencias["TieneIncidencia"] = incidencias["count_incidencias"].apply(lambda x: "Sí" if x > 0 else "No")

incidencias.head()




,id_visit,count_incidencias,TieneIncidencia
0,2555,2,Sí
1,2562,2,Sí
2,2564,2,Sí
3,2572,3,Sí
4,2612,2,Sí


In [8]:
tabla = tabla.merge(incidencias[["id_visit", "TieneIncidencia"]], 
                    on="id_visit", 
                    how="left")

tabla[tabla["id_visit"] == 10485]

,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider,TieneIncidencia
1509,10485,120155808,MISC NONNY,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-25 12:07:07,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN


In [9]:
# Calculo de Motivo
motivo = df_filtrado[df_filtrado["question_id_answer"].isin([8,5])]
motivo = motivo.groupby("id_visit")["answer_answer"].first().reset_index(name="Respuesta_motivo")

motivo.head()


,id_visit,Respuesta_motivo
0,2555,Cliente Cerrado Lona
1,2562,Cliente Cerrado Lona
2,2564,Cliente no permitió cambio
3,2572,Cliente no permitió cambio
4,2612,Cliente no tiene Lona


In [10]:
tabla = tabla.merge(motivo, 
                    on="id_visit", 
                    how="left")

tabla.head()

,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider,TieneIncidencia,Respuesta_motivo
0,48,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,CORPORATIVO MDA,NaN,NaN
1,55,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,CORPORATIVO MDA,NaN,NaN
2,2555,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,Sí,Cliente Cerrado Lona
3,2556,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN
4,2560,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN


In [11]:
# ============= Tamaño Banner / QR
tamaño = df_filtrado[df_filtrado["question_id_answer"].isin([4,14])]
tamaño = tamaño.groupby("id_visit")["answer_answer"].first().reset_index(name="tamaño")

tamaño.head()

,id_visit,tamaño
0,48,Grande
1,55,Grande
2,2556,Básico
3,2560,Grande
4,2561,Básico


In [12]:
tabla = tabla.merge(tamaño,
                    on="id_visit",
                    how = "left")

tabla.head()

,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider,TieneIncidencia,Respuesta_motivo,tamaño
0,48,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,CORPORATIVO MDA,NaN,NaN,Grande
1,55,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,CORPORATIVO MDA,NaN,NaN,Grande
2,2555,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,Sí,Cliente Cerrado Lona,NaN
3,2556,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN,Básico
4,2560,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN,Grande


In [13]:
# ========== Comentario
comentario = df_filtrado[df_filtrado["question_id_answer"].isin([7, 10])]
comentario = comentario.groupby("id_visit")["answer_answer"].first().reset_index(name = "comentario")

tabla = tabla.merge(comentario,
                    on="id_visit",
                    how = "left")

tabla.head()

,id_visit,store_sap_store,store_name_store,store_zone_store,store_region_store,name_type,FechaHora_Menos6h_visit,status_visit,name_provider,TieneIncidencia,Respuesta_motivo,tamaño,comentario
0,48,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,CORPORATIVO MDA,NaN,NaN,Grande,NaN
1,55,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,CORPORATIVO MDA,NaN,NaN,Grande,NaN
2,2555,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,Sí,Cliente Cerrado Lona,NaN,NaN
3,2556,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN,Básico,NaN
4,2560,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN,Grande,NaN


In [14]:
tabla_final = tabla[[
    "store_sap_store",
    "store_name_store",
    "store_zone_store",
    "store_region_store",
    "name_type",
    "FechaHora_Menos6h_visit",
    "status_visit",
    "TieneIncidencia",
    "Respuesta_motivo",
    "name_provider",
    "tamaño",
    "comentario"
]]

In [15]:
# --- Seleccionar y renombrar columnas finales ---
tabla_final.rename(columns={
    "store_sap_store": "SAP",
    "store_name_store": "PDV",
    "store_zone_store": "ZONA",
    "store_region_store": "REGION",
    "name_type": "CATEGORÍA",
    "FechaHora_Menos6h_visit": "FECHA VISITA",
    "status_visit": "ESTADO",
    "TieneIncidencia": "TIENE INCIDENCIA",
    "Respuesta_motivo": "MOTIVO",
    "name_provider": "PROVEEDOR",
    "tamaño": "TAMAÑO",
    "comentario": "COMENTARIO"
})

,SAP,PDV,ZONA,REGION,CATEGORÍA,FECHA VISITA,ESTADO,TIENE INCIDENCIA,MOTIVO,PROVEEDOR,TAMAÑO,COMENTARIO
0,120245260,HERMANOS GARCIA,Valle de Mexico,Reyes,Lona,2025-09-04 08:11:30,Finalizado,NaN,NaN,CORPORATIVO MDA,Grande,NaN
1,110099801,MISC ERICK,Valle de Mexico,Reyes,Lona,2025-09-08 07:44:23,Finalizado,NaN,NaN,CORPORATIVO MDA,Grande,NaN
2,120361697,ABARROTES TIRRENO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 20:21:47,Finalizado,Sí,Cliente Cerrado Lona,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,NaN
3,120238264,ABT BETHEL,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-28 16:03:51,Finalizado,NaN,NaN,JONATHAN ALBERTO LOPEZ MIRANDA,Básico,NaN
4,120281693,ABTS ATENEA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-09-02 13:30:41,Finalizado,NaN,NaN,JONATHAN ALBERTO LOPEZ MIRANDA,Grande,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1641,101002602,MISC ROSITA,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-27 10:30:06,Finalizado,NaN,NaN,JONATHAN ALBERTO LOPEZ MIRANDA,Grande,NaN
1642,120290750,MISC TU Y YO,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-29 10:04:14,Finalizado,NaN,NaN,JONATHAN ALBERTO LOPEZ MIRANDA,Grande,NaN
1643,120301264,TIENDA BEATRIZ,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-31 15:03:23,Finalizado,NaN,NaN,JONATHAN ALBERTO LOPEZ MIRANDA,Grande,NaN
1644,120355365,TIENDA NANCY,Valle de Mexico,Fresno-Iztacalco,Lona,2025-08-29 19:36:44,Finalizado,Sí,Cliente no permitió cambio,JONATHAN ALBERTO LOPEZ MIRANDA,NaN,No sabe de promoción


In [16]:
tabla["status_visit"].value_counts()


status_visit
Finalizado     1644
En progreso       1
Pendiente         1
Name: count, dtype: int64

In [17]:
tabla["status_visit"].unique()


array(['Finalizado', 'En progreso', 'Pendiente'], dtype=object)

In [18]:
df_inc = (
    tabla
    .groupby("status_visit")["id_visit"]
    .nunique()
    .reset_index()
    .rename(columns={"id_visit": "total"})
)

print(df_inc)

  status_visit  total
0  En progreso      1
1   Finalizado   1644
2    Pendiente      1


In [19]:
import psycopg2
import pandas as pd
import streamlit as st

In [20]:
#def get_connection():
#    conn = psycopg2.connect(
#        host=
#        database=
#        password
#        port
#    )
#    return conn

In [21]:
from connection import get_connection
import pandas as pd

In [ ]:
def load_data(query):
    conn = get_connection()
    df = pd.read_sql(query, conn)
    conn.close()
    return df

# ================ PROVEEDORES =========================

query = "SELECT id, code, name, email, status FROM providers;"
providers = load_data(query)
providers = providers.add_suffix("_provider")

# ================ USUARIOS =========================
query2 = """
SELECT id, username, email, full_name, status, role, provider_id
FROM users
WHERE email NOT LIKE '1%'
  AND email NOT LIKE '2%'
  AND email NOT LIKE 'test%';
"""
users = load_data(query2)
users = users.add_suffix("_user")

# ================ VISITAS =========================

query7 = """
SELECT id, status, started_at, ended_at, created_at, updated_at, store_id, user_id, type_id, time_period_id
FROM visits

"""
visits = load_data(query7)
visits['status'] = visits['status'].replace({
    'pending': 'Pendiente',
    'completed': 'Finalizado',
    'progress': 'En progreso'
})

# Asegurarse de que 'updated_at' sea tipo datetime
visits['updated_at'] = pd.to_datetime(visits['updated_at'])

visits['fecha'] = visits['updated_at'].dt.date

# Crear la nueva columna restando 6 horas
visits['FechaHora_Menos6h'] = visits['updated_at'] - pd.Timedelta(hours=6)

visits['Fecha_Menos6h'] = visits['FechaHora_Menos6h'].dt.date
visits = visits.add_suffix("_visit")

#====================== ANSWERS ====================
query8 = """
SELECT id, answer, visit_id, question_id
FROM answers
"""
answers = load_data(query8)
answers['Personalizado'] = np.where(
    (answers['answer'] == "Cliente Cerrado") & (answers['question_id'] == 8),
    "Cliente Cerrado Lona",
    np.where(
        (answers['answer'] == "Cliente Cerrado") & (answers['question_id'] == 5),
        "Cliente Cerrado Banner + Rack",
        answers['answer']
    )
)
answers = answers.rename(columns={
    'answer': 'answer1',
    'Personalizado': 'answer'
})
answers = answers[['id', 'answer', 'visit_id', 'question_id']]  # reemplaza con las que quieras

answers['answer'] = np.where(
    answers['answer'].str.contains('Basic', case=False, na=False),
    'Básico',
    answers['answer']
)

answers['answer'] = np.where(
    answers['answer'].str.contains('Extra\s?Large', case=False, na=False, regex=True),
    'Extragrande',
    answers['answer']
)

answers['answer'] = np.where(
    answers['answer'].str.contains('Large', case=False, na=False),
    'Grande',
    answers['answer']
)

answers['answer'] = answers['answer'].replace(
    'Cliente no tiene Lona', 
    'Cliente no tiene Banner'
)
answer = answers.add_suffix("_answer")
# ================= TAMAÑOS DE VISITA ================================

# Extraer tamaños por visita
temp = answer[answer["question_id_answer"].isin([4, 14])][
    ["visit_id_answer", "answer_answer"]
].rename(columns={
    "visit_id_answer": "id_visit",
    "answer_answer": "TamañoAsignado_answer"
})

# Unirlo a visits
visits = visits.merge(temp, on="id_visit", how="left")

#==================== TIME PERIODS ===========================
query7 = """
SELECT id, name
FROM time_periods
"""
periods = load_data(query7)
periods = periods.add_suffix("_period")


# ==================== STORES =============
query6 = """
SELECT id, store_sap, store_name, store_zone, store_region, store_mr,  status
FROM stores

"""
stores = load_data(query6)
stores = stores.add_suffix("_store")

# ================= QUESTIONS ================================

query5 = """
SELECT id, question, section_id
FROM questions

"""
questions = load_data(query5)
questions = questions.add_suffix("_question")



#===================== SECTIONS ===============================

query4 = """
SELECT id, section_name, status, type_id
FROM sections

"""
sections = load_data(query4)
sections = sections.add_suffix("_section")

# ========================= TYPES ===========================
query3 = """
SELECT id, name, status
FROM types

"""
types = load_data(query3)
types = types.add_suffix("_type")

#=================== COMBINAR TABLAS ============================
df_combined = providers.merge(users, left_on='id_provider', right_on='provider_id_user', how='inner')
df_combined = df_combined.merge(visits, left_on='id_user', right_on='user_id_visit', how='inner')
df_combined = df_combined.merge(periods, left_on='time_period_id_visit', right_on='id_period', how='left')
df_combined = df_combined.merge(answer, left_on='id_visit', right_on='visit_id_answer', how='left')
df_combined = df_combined.merge(stores, left_on='store_id_visit', right_on='id_store', how='inner')
df_combined = df_combined.merge(questions, left_on='question_id_answer', right_on='id_question', how='left')
df_combined = df_combined.merge(sections, left_on='section_id_question', right_on='id_section', how='left')
df_combined = df_combined.merge(types, left_on='type_id_visit', right_on='id_type', how='inner')
df_combined.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_8728\3213963824.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_8728\3213963824.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


DatabaseError: Execution failed on sql '
SELECT id, answer, visit_id, question_id,sectionid
FROM answers
': column "sectionid" does not exist
LINE 2: SELECT id, answer, visit_id, question_id,sectionid
                                                 ^


In [ ]:
df_combined.columns

Index(['id_provider', 'code_provider', 'name_provider', 'email_provider',
       'status_provider', 'id_user', 'username_user', 'email_user',
       'full_name_user', 'status_user', 'role_user', 'provider_id_user',
       'id_visit', 'status_visit', 'started_at_visit', 'ended_at_visit',
       'created_at_visit', 'updated_at_visit', 'store_id_visit',
       'user_id_visit', 'type_id_visit', 'time_period_id_visit', 'fecha_visit',
       'FechaHora_Menos6h_visit', 'Fecha_Menos6h_visit', 'TamañoAsignado',
       'id_period', 'name_period', 'id_answer', 'answer_answer',
       'visit_id_answer', 'question_id_answer', 'id_store', 'store_sap_store',
       'store_name_store', 'store_zone_store', 'store_region_store',
       'store_mr_store', 'status_store', 'id_question', 'question_question',
       'section_id_question', 'id_section', 'section_name_section',
       'status_section', 'type_id_section', 'id_type', 'name_type',
       'status_type'],
      dtype='object')

In [24]:
# Asegurar que updated_at es datetime
visits['updated_at_visit'] = pd.to_datetime(visits['updated_at_visit'])

# Duplicar la columna
visits['updated_at_date'] = visits['updated_at_visit'].dt.date

In [25]:
visits['updated_at_date']

0         2025-07-22
1         2025-08-25
2         2025-09-04
3         2025-09-04
4         2025-09-04
             ...    
177177    2025-11-19
177178    2025-11-19
177179    2025-11-19
177180    2025-11-19
177181    2025-11-19
Name: updated_at_date, Length: 177182, dtype: object